In [10]:
import pandas as pd
import numpy as np
from sklearn import svm, neighbors,tree, ensemble, model_selection, preprocessing
from sklearn.model_selection import train_test_split


In [11]:
df = pd.read_csv('fulldata_colonnes_essentielles_prepross.csv')



In [12]:
# Les lignes ayant le même 'Accession' doivent être comprises comme les données d'une même entité (code Amal)
np.random.seed = 12

unique_users = df['ACCESSION'].unique()
target_n = int(.8 * len(df))
shuffled_users = np.random.permutation(unique_users)

user_count = df['ACCESSION'].value_counts()

mapping = user_count.reindex(shuffled_users).cumsum() <= target_n
mask = df['ACCESSION'].map(mapping)

df_train = df[mask]
df_test = df[~mask]

df_train.head(4)

,Unnamed: 0,ACCESSION,class1,class2,class3,AC_mode,ion_precursor_mz,num_peak,peak_mz,peak_rel_int,peak_int
0,7,PM000102,Phenylpropanoid,Phenylpropanoid glycoside,Phenylpropanoid glycoside,0,0.140564,0.122807,0.063692,0.002695,0.0
1,8,PM000102,Phenylpropanoid,Phenylpropanoid glycoside,Phenylpropanoid glycoside,0,0.140564,0.122807,0.037912,0.008985,0.0
2,9,PM000102,Phenylpropanoid,Phenylpropanoid glycoside,Phenylpropanoid glycoside,0,0.140564,0.122807,0.034229,0.089847,0.0
3,10,PM000102,Phenylpropanoid,Phenylpropanoid glycoside,Phenylpropanoid glycoside,0,0.140564,0.122807,0.030329,0.053908,0.0


In [13]:
#Séparation en variables cibles et explicatives :  chaque classe est une variable cible différente.
#La classe la plus interessante a priori est la classe 3, la plus 'précise' (classe 3 incluse dans la 2 qui est incluse dans la 1)
 
X_test = df_test[['AC_mode', 'ion_precursor_mz', 'peak_mz', 'peak_rel_int', 'num_peak']]
y_test3 = df_test.class3
y_test2 = df_test.class2
y_test1 = df_test.class1

X_train = df_train[['AC_mode', 'ion_precursor_mz', 'peak_mz', 'peak_rel_int', 'num_peak']]
y_train3 = df_train.class3
y_train2 = df_train.class2
y_train1 = df_train.class1

In [14]:
#classification paramètre par défaut rf CLASS3
clf_rf = ensemble.RandomForestClassifier(random_state=321)
clf_rf.fit(X_train, y_train3)

clf_rf.predict(X_test)
print('Score de rf:', clf_rf.score(X_test, y_test3))

Score de rf: 0.4938812170203211


In [15]:
#classification paramètre par défaut rf CLASS2
clf_rf = ensemble.RandomForestClassifier(random_state=321)
clf_rf.fit(X_train, y_train2)

clf_rf.predict(X_test)
print('Score de rf:', clf_rf.score(X_test, y_test2))

Score de rf: 0.5732569888851465


In [16]:
#classification paramètre par défaut rf CLASS1
clf_rf = ensemble.RandomForestClassifier(random_state=321)
clf_rf.fit(X_train, y_train1)

clf_rf.predict(X_test)
print('Score de rf:', clf_rf.score(X_test, y_test1))

Score de rf: 0.6926013248007186


In [7]:
#recherche des meilleurs paramètres forêt aléatoire CLASS3

clf_rf = ensemble.RandomForestClassifier()

parametres_rf = {}

grid_clf_rf = model_selection.GridSearchCV(estimator = clf_rf, param_grid = parametres_rf)

grille = grid_clf_rf.fit(X_train, y_train3)

print('Meilleurs paramètres de rf:', grid_clf_rf.best_params_)

y_pred_rf = grid_clf_rf.predict(X_test)

print('Score de rf:', grid_clf_rf.score(X_test, y_test3))